Sigh. Zoe Bjeffs takes the game with 1046 points. Impressive, but your strategy seems to have failed this time too. It seems impossible to make your CEO win.

Thinking about how your CEO bowls, maybe something that rewards consecutive strikes could be a good idea. You think for a while and come up with these rule changes:

* Each strike rewards the next two rolls as bonus points.
* Each spare rewards the next roll as bonus points.

They feel familiar somehow, but you quickly dismiss that thought and move on to the example:

```
Yatas Del Lana | 3 5 | 3 5 | 7 2 | 3 0 | 10  | 4 3
Eve Stojbs     | 3 7 | 3 3 | 9 1 | 6 4 | 2 3 | 1 0
```

Summing up the scores according to these new rules you arrive at:

* Yatas Delana: 3 + 5 + 3 + 5 + 7 + 2 + 3 + 0 + 10 + (4 + 3) (Strike) + 4 + 3 = 52
* Eve Stojbs: 3 + 7 + 3 (Spare) + 3 + 3 + 9 + 1 + 6 (Spare) + 6 + 4 + 2 (Spare) + 2 + 3 + 1 + 0 = 53 

And according to these rules Eve Stojbs wins with 53 points.

You send your email with the proposed rule changes and hope for the best.

Given the scores in the new series, calculate the winner and their score. 

In [0]:
from pyspark.sql.functions import when, sum, col, lead, coalesce
from pyspark.sql.window import Window

frames = spark.table("ceo_bowling_frames").where("Game = 4")
players = spark.table("ceo_bowling_players")
frames_with_score = frames \
  .withColumn("Score", frames.Throw1 + when(frames.Throw2.isNull(), 0).otherwise(frames.Throw2)) \
  .withColumn("Strike", when(frames.Throw1 == 10, True).otherwise(False)) \
  .withColumn("Spare", when(frames.Throw1 + frames.Throw2 == 10, True).otherwise(False))

window_spec = Window.partitionBy(frames.PlayerID).orderBy("Frame")
bonus_scores = frames_with_score \
    .withColumn("BonusScore",
                when(frames_with_score.Strike, lead(frames.Throw1).over(window_spec)
                     + coalesce(lead(frames.Throw2).over(window_spec), lead(frames.Throw1, offset=2).over(window_spec)))
                .when(frames_with_score.Spare, lead(frames.Throw1).over(window_spec))
                .otherwise(0))
final_scores = bonus_scores \
    .groupBy("PlayerID") \
    .agg(sum(bonus_scores.Score + when(bonus_scores.BonusScore.isNotNull(), bonus_scores.BonusScore).otherwise(0)).alias("Total")) \
    .join(players, ["PlayerID",]) \
    .orderBy(col("Total").desc())

display(bonus_scores)
display(final_scores)